<a href="https://colab.research.google.com/github/FredSeph/Alura_WonderWorld/blob/main/WonderWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WonderWorld**

Projeto criado por Fredson Ricelli Santos em 09/05/2024, para o desafio da Imersão IA_ da Alura e Google.

Start:

In [55]:
!pip install -q -U google-generativeai

import google.generativeai as genai
import requests
import json
import ipywidgets as widgets

from google.colab import userdata

api_key = userdata.get("gemini_api_key")

genai.configure(api_key=api_key) # Use sua própria api key do gemini aqui.

generation_config = {
    "candidate_count": 1,
    "temperature": 0.5
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [ ]:
#Criar o input

place = input("Digite o nome de um lugar: ")

#Criação do prompt inicial e obtenção da resposta com um json com lugares possíveis

prompt_inicial = f"""
Busque lugares no mundo com o nome '{place} ou que o nome contenha '{place}'.

Objetivo: Buscar lugares no mundo com o nome fornecido, retornando apenas lugares que tenha as seguintes informações: nome, cidade, estado e país.
Se alguma dessas informações não estiver disponível, o lugar não deve ser incluído na lista.
Busque o máximo de lugares possíveis.

A resposta deve ser em formato JSON.

Instruções para a geração de dados JSON:

Formato da Resposta: JSON estrito, sem caracteres extras ou marcações.

Estrutura do JSON:

{{
  'count': <número_de_resultados>,
  'items': [
    {{
      'name': '<nome_do_lugar>',
      'city': '<nome_da_cidade>',
      'state': '<nome_do_estado>',
      'country': '<nome_do_país>'
    }},
    ...
  ]
}}

Restrições:

* Itens Completos: Todos os itens na lista 'items' devem ter valores válidos em todas as propriedades: 'name', 'city', 'state' e 'country'. Nenhuma pode ter 'None' como valor.
* Excluir Itens Incompletos: Se um lugar não possui informações para todas as quatro propriedades, ele NÃO deve ser incluído na lista 'items'.

Exemplo de Resposta Válida:

{{
  'count': 1,
  'items': [
    {{
      'name': 'Belo Horizonte',
      'city': 'Belo Horizonte',
      'state': 'Minas Gerais',
      'country': 'Brazil'
    }}
  ]
}}

É estritamente necessário que a resposta não contenha nada além do json. Envie a resposta sem '```json' e ```.
"""

response = model.generate_content(prompt_inicial)

data = json.loads(response.text)

# Método que cria e retorna o prompt final, dado o lugar escolhido

def get_prompt_final(place, city, state, country):

  prompt_final = f"""Estou planejando uma viagem para **[{place}, {city}, {state}, {country}]**.

  **Informações que desejo receber:**

  * **Melhor época do ano para visitar:**
    * Meses (de [mes inicial] até [mes final])
    * Motivos para visitar nessa época (limite de 1000 caracteres)

  * **Curiosidade sobre o destino:**
    * Limite de 2000 caracteres

  * **Previsão do tempo em {city}:**
    * **[Resumo da previsão do tempo nesse período sugerido, se disponível.]**
    * Se não tiver dados sobre a previsão do tempo na época sugerida para visita, informe que não há dados sobre a previsão do tempo *

  **Formato de saída:**

  * Local: [Local]
  * Cidade: [Cidade]
  * Estado: [Estado]
  * País: [País]

  **Melhor época para visitar [Local]:**

  * Melhor época para visitar [Local] é entre [mês inicial] e [mês final].
  * [Motivos para visitar nessa época, em formato de lista]

  **Curiosidade sobre [Local]:**

  [Curiosidade sobre [Local]]

  **Previsão do tempo:**

  [Previsão do tempo]"""

  return prompt_final



options = []

count_resultados = data['count']

if count_resultados > 0:

  if count_resultados > 1:

    # Criação de options para um dropdown caso o resultado do prompt inicial tenha mais de um item na lista

    options.append(('Escolha um lugar...', ''))

    for p in data['items']:

      # Cria o texto do option e armazena informações do lugar

      texto = f"{p['name']} - {p['city']} {p['state']}/{p['country']}"
      options.append((texto, p))

    # Cria o dropdown

    dropdown = widgets.Dropdown(options=options, description='Escolha um lugar:')

    # Define a execução do click do botão

    def on_change(change):

      if change['type'] == 'change' and change ['name'] == 'value':
        chosen_place = change['new']
        prompt_final = get_prompt_final(chosen_place['name'], chosen_place['city'], chosen_place['state'], chosen_place['country'])
        response = model.generate_content(prompt_final)
        print(f"\n\n{response.text}")

    # Associa a função on_change ao evento de mudança do Dropdown

    dropdown.observe(on_change)

    # Exibir os botões

    display(dropdown)

  else:

    # Caso tenha apenas um item no json, seleciona e cria o prompt final direto

    chosen_place = data['items'][0]

    prompt_final = get_prompt_final(chosen_place['name'], chosen_place['city'], chosen_place['state'], chosen_place['country'])
    response = model.generate_content(prompt_final)

    print(f"\n\n{response.text}")

else:

  print("Não foram encontrados lugares com este nome.")

O projeto consiste em dar o nome de um lugar, e obter informações como a melhor época para visitar, curiosidades, e previsão do tempo na época sugerida. Se existe mais de um lugar contendo o nome digitado, exibirá um dropdown com as opções disponíveis. Ao clicar em uma opção, retorna as informações.

Observação: Por mais que eu tenha insistido no prompt inicial de diversas formas para não incluir itens na lista quando não for possível obter todas as propriedades, ainda pode ocorrer do modelo incluir estes itens na lista, com propriedades com valor 'None'.

Criei um "Helper" logo abaixo, para não precisar ficar mudando de aba para fazer perguntas ao Gemini. Estava usando para me auxiliar na construção do projeto. Este Helper foi criado baseado no que foi ensinado na Aula 4, mas fiz algumas modificações como substituir o input para um textarea para melhor visualização quando os prompts forem grandes. Também deixei o código para exibir histórico, da Aula 4.

Helper:

In [ ]:
from ipywidgets import Textarea, Button

chat = model.start_chat(history=[])

prompt = Textarea(value='',
                  placeholder='Insira o seu prompt.',
                  disabled=False)
button = Button(description="Enviar")

def on_button_click(b):
  response = chat.send_message(prompt.value)
  print(f"Gemini: {response.text} \n")

  prompt.value = ''
  button.on_click(on_button_click)

button.on_click(on_button_click)

display(prompt, button)

Textarea(value='', placeholder='Insira o seu prompt.')

Button(description='Enviar', style=ButtonStyle())

Gemini: **Prompt:**

Estou planejando uma viagem de **[Origem]** para **[Destino]** em **[Data]**.

**Informações que desejo receber:**

* **Opção de voo mais barata:**
    * Companhia aérea
    * Data e hora do voo
    * Preço

* **Curiosidade sobre o destino:**
    * Limite de 1000 caracteres

* **Previsão do tempo no destino:**
    * Data: **[Data]**

**Restrições:**

* Se o destino ou a data forem inválidos, retorne uma mensagem de erro.
* Se a data for anterior à data atual, retorne uma mensagem de erro.

**Formato de saída:**

```
**Opção de voo mais barata:**

* Companhia aérea: [Nome da companhia aérea]
* Data e hora do voo: [Data e hora]
* Preço: [Preço]

**Curiosidade sobre o destino:**

[Curiosidade sobre o destino]

**Previsão do tempo:**

[Previsão do tempo]
``` 



Histórico:

In [ ]:
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')